In [2]:
from IPython.core.display import display, HTML

import pandas as pd
import numpy as np
from scipy import stats
import random
import glob
import os
import gc

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.optim as optim
import torch.nn.functional as F
from torch.utils import data

from joblib import Parallel, delayed

from sklearn import preprocessing, model_selection
from tqdm import tqdm

import matplotlib.pyplot as plt
import seaborn as sns

path_root = './'
data_dir ='./'
path_submissions = '/'

target_name = 'target'

DEBUG = False


In [3]:
def log_return(list_stock_prices):
    return np.log(list_stock_prices).diff() 

def realized_volatility(series_log_return):
    return np.sqrt(np.sum(series_log_return**2))

def realized_mad(series_log_return):
    return np.mean(np.absolute(series_log_return - np.mean(series_log_return)))

def realized_median_abs_dev(series_log_return):
    return stats.median_absolute_deviation(series_log_return, nan_policy='omit')

def rmspe(y_true, y_pred):
    return  (np.sqrt(np.mean(np.square((y_true - y_pred) / y_true))))

def calc_wap(df):
    wap = (df['bid_price1'] * df['ask_size1'] + df['ask_price1'] * df['bid_size1'])/(df['bid_size1'] + df['ask_size1'])
    return wap

def calc_wap2(df):
    wap = (df['bid_price2'] * df['ask_size2'] + df['ask_price2'] * df['bid_size2'])/(df['bid_size2'] + df['ask_size2'])
    return wap

def count_unique(series):
    return len(np.unique(series))

In [4]:
def preprocessor_book(file_path):
    df = pd.read_parquet(file_path)
    
    df['wap'] = calc_wap(df)
    df['log_return'] = df.groupby('time_id')['wap'].apply(log_return)
    
    df['wap2'] = calc_wap(df)
    df['log_return2'] = df.groupby('time_id')['wap2'].apply(log_return)
    
    df['wap_imbalance'] = abs(df['wap'] - df['wap2'])
    
    df['spread'] = (df['ask_price1'] - df['bid_price1']) / ((df['ask_price1'] + df['bid_price1'])/2)
    
    df['bid_spread'] = df['bid_price1'] - df['bid_price2']
    df['ask_spread'] = df['ask_price1'] - df['ask_price2']
    df['total_volume'] = (df['ask_size1'] + df['ask_size2']) + (df['bid_size1'] + df['bid_size2'])
    df['volume_imbalance'] = abs((df['ask_size1'] + df['ask_size2']) - (df['bid_size1'] + df['bid_size2']))
    
    agg_dict = {
        'log_return':[realized_volatility,realized_mad,realized_median_abs_dev],
        'log_return2':[realized_volatility,realized_mad,realized_median_abs_dev],
        'wap_imbalance':[np.mean],
        'spread':[np.mean],
        'bid_spread':[np.mean],
        'ask_spread':[np.mean],
        'volume_imbalance':[np.mean],
        'total_volume':[np.mean],
        'wap':[np.mean],
    }
    
    
    df_feature = pd.DataFrame(df.groupby(['time_id']).agg(agg_dict)).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns] #time_id is changed to time_id_
        
    #create row_id
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['time_id_'],axis=1)
    
    return df_feature
    

In [5]:
%%time
file_path = data_dir + "book_train.parquet/stock_id=0"
preprocessor_book(file_path)

Wall time: 5.84 s


,log_return_realized_volatility,log_return_realized_mad,log_return_realized_median_abs_dev,log_return2_realized_volatility,log_return2_realized_mad,log_return2_realized_median_abs_dev,wap_imbalance_mean,spread_mean,bid_spread_mean,ask_spread_mean,volume_imbalance_mean,total_volume_mean,wap_mean,row_id
0,0.004499,0.000157,0.000053,0.004499,0.000157,0.000053,0.0,0.000852,0.000176,-0.000151,134.894040,323.496689,1.003725,0-5
1,0.001204,0.000038,0.000005,0.001204,0.000038,0.000005,0.0,0.000394,0.000142,-0.000135,142.050000,411.450000,1.000239,0-11
2,0.002369,0.000092,0.000020,0.002369,0.000092,0.000020,0.0,0.000725,0.000197,-0.000198,141.414894,416.351064,0.999542,0-16
3,0.002574,0.000113,0.000012,0.002574,0.000113,0.000012,0.0,0.000860,0.000190,-0.000108,146.216667,435.266667,0.998832,0-31
4,0.001894,0.000068,0.000011,0.001894,0.000068,0.000011,0.0,0.000397,0.000191,-0.000109,123.846591,343.221591,0.999619,0-62
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3825,0.002579,0.000078,0.000016,0.002579,0.000078,0.000016,0.0,0.000552,0.000083,-0.000182,197.144781,374.235690,0.997938,0-32751
3826,0.002206,0.000063,0.000001,0.002206,0.000063,0.000001,0.0,0.000542,0.000092,-0.000172,233.781553,621.131068,1.000310,0-32753
3827,0.002913,0.000134,0.000075,0.002913,0.000134,0.000075,0.0,0.000525,0.000202,-0.000083,115.829787,343.734043,0.999552,0-32758
3828,0.003046,0.000107,0.000070,0.003046,0.000107,0.000070,0.0,0.000480,0.000113,-0.000166,132.074919,385.429967,1.002357,0-32763


In [6]:
def preprocessor_trade(file_path):
    df = pd.read_parquet(file_path)
    df['log_return'] = df.groupby('time_id')['price'].apply(log_return)
    df['dollar_volume'] = df['price'] * df['size']
    
    
    agg_dict = {
        'log_return':[realized_volatility,realized_mad,realized_median_abs_dev],
        'seconds_in_bucket':[count_unique],
        'size':[np.sum],
        'order_count':[np.mean],
        'dollar_volume':[np.sum],
    }
    
    df_feature = df.groupby('time_id').agg(agg_dict).reset_index()
    
    df_feature.columns = ['_'.join(col) for col in df_feature.columns]

    
    df_feature = df_feature.add_prefix('trade_')
    stock_id = file_path.split('=')[1]
    df_feature['row_id'] = df_feature['trade_time_id_'].apply(lambda x:f'{stock_id}-{x}')
    df_feature = df_feature.drop(['trade_time_id_'],axis=1)
    
    return df_feature

In [7]:
%%time
file_path = data_dir + "trade_train.parquet/stock_id=0"
preprocessor_trade(file_path)

Wall time: 2.81 s


,trade_log_return_realized_volatility,trade_log_return_realized_mad,trade_log_return_realized_median_abs_dev,trade_seconds_in_bucket_count_unique,trade_size_sum,trade_order_count_mean,trade_dollar_volume_sum,row_id
0,0.002006,0.000271,0.000345,40,3179,2.75,3190.139181,0-5
1,0.000901,0.000133,0.000149,30,1289,1.9,1289.353432,0-11
2,0.001961,0.000298,0.000307,25,2161,2.72,2158.608928,0-16
3,0.001561,0.000321,0.000383,15,1962,3.933333,1959.605547,0-31
4,0.000871,0.000138,0.000158,22,1791,4.045455,1790.254496,0-62
...,...,...,...,...,...,...,...,...
3825,0.001519,0.000162,0.000155,52,3450,3.057692,3441.815546,0-32751
3826,0.001411,0.000236,0.000315,28,4547,3.892857,4548.671493,0-32753
3827,0.001521,0.000206,0.000298,36,4250,3.5,4247.563002,0-32758
3828,0.001794,0.000200,0.000227,53,3217,2.150943,3224.421796,0-32763


In [8]:
def preprocessor(list_stock_ids, is_train = True):
    from joblib import Parallel, delayed # parallel computing to save time
    df = pd.DataFrame()
    
    def for_joblib(stock_id):
        if is_train:
            file_path_book = data_dir + "book_train.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_train.parquet/stock_id=" + str(stock_id)
        else:
            file_path_book = data_dir + "book_test.parquet/stock_id=" + str(stock_id)
            file_path_trade = data_dir + "trade_test.parquet/stock_id=" + str(stock_id)
            
        df_tmp = pd.merge(preprocessor_book(file_path_book),preprocessor_trade(file_path_trade),on='row_id',how='left')
     
        return pd.concat([df,df_tmp])
    
    df = Parallel(n_jobs=-1, verbose=1)(
        delayed(for_joblib)(stock_id) for stock_id in list_stock_ids
        )

    df =  pd.concat(df,ignore_index = True)
    return df

In [9]:
train = pd.read_csv(os.path.join(data_dir,'train.csv'))
df_train = pd.read_csv('train_processed.csv')

In [10]:
test = pd.read_csv(os.path.join(data_dir,'test.csv'))
test_ids = test.stock_id.unique()

In [11]:
%%time
df_test = preprocessor(list_stock_ids=test_ids, is_train=False)

[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.


Wall time: 1.05 s


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:    0.9s finished


In [12]:
df_test = test.merge(df_test, on=['row_id'], how='left')

In [13]:
df_train['stock_id'] = df_train['row_id'].apply(lambda x:x.split('-')[0])
df_test['stock_id'] = df_test['row_id'].apply(lambda x:x.split('-')[0])
df_train['time_id'] = df_train['row_id'].apply(lambda x:x.split('-')[1])
df_test['time_id'] = df_test['row_id'].apply(lambda x:x.split('-')[1])

In [14]:
df_train.columns

Index(['row_id', 'target', 'log_return_realized_volatility',
       'log_return_realized_mad', 'log_return_realized_median_abs_dev',
       'log_return2_realized_volatility', 'log_return2_realized_mad',
       'log_return2_realized_median_abs_dev', 'wap_imbalance_mean',
       'spread_mean', 'bid_spread_mean', 'ask_spread_mean',
       'volume_imbalance_mean', 'total_volume_mean', 'wap_mean',
       'trade_log_return_realized_volatility', 'trade_log_return_realized_mad',
       'trade_log_return_realized_median_abs_dev',
       'trade_seconds_in_bucket_count_unique', 'trade_size_sum',
       'trade_order_count_mean', 'trade_dollar_volume_sum', 'stock_id',
       'time_id'],
      dtype='object')

In [15]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

In [34]:
class OptiveDataset(Dataset):
    def __init__(self, X, Y, emb_cols=['stock_id', 'time_id']):
        X = X.copy()
        self.X1 = X.loc[:,emb_cols].copy().values.astype(np.int64) #categorical columns
        self.X2 = X.drop(columns=emb_cols).copy().values.astype(np.float32) #numerical columns
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return (np.expand_dims(self.X1[idx], 0), np.expand_dims(self.X2[idx], 0)), np.expand_dims(self.y[idx], 0)
    
class OptiveDatasetTest(Dataset):
    def __init__(self, X, emb_cols=['stock_id', 'time_id']):
        X = X.copy()
        self.X1 = X.loc[:,emb_cols].copy().values.astype(np.int64) #categorical columns
        self.X2 = X.drop(columns=emb_cols).copy().values.astype(np.float32) #numerical columns
        
    def __len__(self):
        return len(self.X1)
    
    def __getitem__(self, idx):
        return (self.X1[idx], self.X2[idx])

In [17]:
df_train = df_train.fillna(0)
df_test = df_test.fillna(0)

In [55]:
train_dataset = OptiveDataset(df_train.drop(['target', 'time_id','row_id'], axis=1), df_train['target'], emb_cols=['stock_id'])
train_dl = DataLoader(train_dataset, batch_size=4, shuffle=True)

#test the dataset class
for (emb, cont), target in train_dl:
    print((emb.shape, cont.shape), target.shape)
    break;

(torch.Size([4, 1, 1]), torch.Size([4, 1, 20])) torch.Size([4, 1])


In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# device = torch.device('cpu')

def RMSELoss(yhat,y):
    return torch.sqrt(torch.mean((yhat-y)**2))

def RMSPELoss(y_pred, y_true):
    return torch.sqrt(torch.mean( ((y_true - y_pred) / y_true) ** 2 ))

In [46]:
class OptiverModel(nn.Module):
    # d_model : number of features
    def __init__(self,feature_size=7,num_layers=3,dropout=0, embedding_sizes=16, num_embeddings=max(df_train['stock_id'].astype(np.int8))+1):
        super(OptiverModel, self).__init__()
        
        self.emb = nn.Embedding(num_embeddings, embedding_sizes)
        self.emb_drop = nn.Dropout(0.25)
        self.bn1 = nn.BatchNorm1d(20)
        
        self.encoder_layer = nn.TransformerEncoderLayer(d_model=feature_size, nhead=7, dropout=dropout)
        self.transformer_encoder = nn.TransformerEncoder(self.encoder_layer, num_layers=num_layers)        
        self.decoder = nn.Linear(feature_size,1)
        self.init_weights()

    def init_weights(self):
        initrange = 0.1    
        self.decoder.bias.data.zero_()
        self.decoder.weight.data.uniform_(-initrange, initrange)

    def _generate_square_subsequent_mask(self, sz):
        mask = (torch.triu(torch.ones(sz, sz)) == 1).transpose(0, 1)
        mask = mask.float().masked_fill(mask == 0, float('-inf')).masked_fill(mask == 1, float(0.0))
        return mask

    def forward(self, x_cat, x_cont, device):
        x1 = self.emb(x_cat)
        x1 = torch.flatten(x1, end_dim=1)
        x1 = self.emb_drop(x1)
        x2 = x_cont.permute(0,2,1)
        x2 = self.bn1(x2)
#         x2 = x2.permute(0,2,1)
        x1 = x1.permute(0,2,1)
        x = torch.cat([x1,x2], 1)
        x = x.permute(0,2,1)
        
        mask = self._generate_square_subsequent_mask(len(x)).to(device)
        output = self.transformer_encoder(x,mask)
        output = self.decoder(output)
        return output

In [30]:
def train_epoch(train_dl, valid_dl, model, loss_fn, opt, sch, epoch, fold, device=device):
    # taining loop
    model.train()
    running_loss_ = 0
    
    pbar = tqdm(enumerate(train_dl), total=len(train_dl))
    for i, ((cats, counts), targets) in pbar:
        cats, counts, targets = cats.to(device), counts.to(device), targets.unsqueeze(1).to(device)
        
        opt.zero_grad()
        y_pred = model(cats, counts, device)
        loss = loss_fn(y_pred.float(), targets.float())
        
        loss.backward()
        opt.step()
        
        running_loss_ += loss.item()
        if (i+1) % 100 == 0:
            pbar.set_description(f"running loss:{running_loss_ / (i+1): 0.6f}")
    
    sch.step(loss)

    epoch_loss = running_loss_ / len(train_dl)
    #print(f'==> Epoch {epoch} TRAIN loss: {epoch_loss:.6f}')
    
    # Validation loop
    model.eval()
    valid_loss = 0
    best_loss = np.inf
    
    for i, ((cats, counts), targets) in enumerate(valid_dl):
        cats, counts, targets = cats.to(device), counts.to(device), targets.unsqueeze(1).to(device)
        
        with torch.no_grad():
            y_pred = model(cats, counts, device)
            val_loss = loss_fn(y_pred.float(), targets.float())
            
        valid_loss += val_loss.item() * targets.shape[0]
    sch.step(valid_loss)
    
    valid_epoch_loss = valid_loss / len(valid_dl)
    print(f'==>F{fold}, Epoch {epoch} VALID loss: {valid_epoch_loss:.8f}')
    
    if valid_epoch_loss < best_loss:
        best_loss = valid_epoch_loss
        torch.save(model.state_dict(), f'FOLD{fold}_optive_model.pth')
    
    model.train()
    return model, epoch_loss, valid_epoch_loss

In [23]:
def perpare_dataset(train, valid, test=None, batch_size=64, drop_cols=['target', 'time_id', 'row_id'], emb_cols=['stock_id']):
    train_dataset = OptiveDataset(train.drop(drop_cols, axis=1), train['target'], emb_cols=emb_cols)
    valid_dataset = OptiveDataset(valid.drop(drop_cols, axis=1), valid['target'], emb_cols=emb_cols)    
    
    train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    valid_dl = DataLoader(valid_dataset, batch_size=batch_size, shuffle=True)
    
    return train_dl, valid_dl

# Training Loop

Conducted remotely on GCP instance - Transformers need more compute!

In [49]:
n_folds = 10
epochs = 10

kf = model_selection.KFold(n_splits=n_folds, shuffle=True, random_state=42)
seed_everything(46)

for fold_idx, (dev_index, val_index) in enumerate(kf.split(range(len(train)))):
    
    if fold_idx > 4:
        break #train 5 folds
        
    train_ = df_train.loc[dev_index,].reset_index(drop=True)
    valid_ = df_train.loc[val_index, ].reset_index(drop=True)
    
    train_dl, valid_dl = perpare_dataset(train_, valid_)
    
    model = OptiverModel(feature_size=49, dropout=.25, embedding_sizes=29,).to(device)
    loss_fn = RMSELoss
    
    opt = optim.Adam(model.parameters(), lr=0.01)
    sch = optim.lr_scheduler.ReduceLROnPlateau(opt, factor=0.2, patience=3)
    
    counter = 0
    for epoch in range(epochs):
        model, epoch_loss, valid_epoch_loss = train_epoch(train_dl, valid_dl, 
                                                                   model, loss_fn, opt, 
                                                                   sch, epoch, fold_idx, device=device)

  0%|                                                                                 | 8/6032 [00:00<01:23, 72.07it/s]

==>F0, Epoch 0 VALID loss: 0.19428741


running loss: 0.003016:  42%|██████████████████████▍                               | 2508/6032 [00:30<00:42, 83.38it/s]


KeyboardInterrupt: 